In [1]:
import scipy.io as scio 
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import matplotlib.cbook as cbook
from sklearn.cluster import KMeans
import spectral as sp
import numpy as np
import glob
import os 
from sklearn.cluster import KMeans
import skimage as ski 
import skfuzzy as skf

In [3]:
pavia = {'rgb': scio.loadmat('PaviaRGB.mat')['PaviaRGB'],
         'color_map' : scio.loadmat('PaviaColorMap.mat')['PaviaColorMap'],
         'ground_truth' : scio.loadmat('PaviaGrTruth.mat')['PaviaGrTruth'],
         'ground_truth_mask' : scio.loadmat('PaviaGrTruthMask.mat')['PaviaGrTruthMask'],
         'hyperimage' : scio.loadmat('PaviaHyperIm.mat')['PaviaHyperIm'],
         'wavelengths' : scio.loadmat('PaviaWavelengths.mat')['PaviaWavelengths'],
        }
sanbar = {'rgb': scio.loadmat('SanBarRGB.mat')['SanBarRGB'],
          'wavelengths': scio.loadmat('SanBarWavelengths.mat')['WaveLengths'],
          'hyperimage': scio.loadmat('SanBarHyperIm.mat')['SanBarIm88x400'],
         }
def k_means_clustering(inp=pavia['rgb'], n_clusters=2):
    if inp is None:
        print("Empty Input. Exiting")
    k_means = KMeans(n_clusters=n_clusters)
    shape = inp.shape
    k_means.fit(inp.flatten().reshape(shape[0]*shape[1],shape[2]))
    inp_clustered = k_means.labels_.astype(float)
    return inp_clustered.reshape(shape[0]*shape[1],1).reshape(shape[0],shape[1])

def fuzzy_c_means(n_clusters=2,fuzz_degree=2,error=0.005,max_iter=1000,inp_image=pavia['rgb']):
    shape = inp_image.shape
    centers, u, u0, d, jm, n_iters, fpc = skf.cluster.cmeans(
        inp.flatten().reshape(shape[0]*shape[1],shape[2]).T,
        c=n_clusters,
        m=fuzz_degree,
        error=error,
        maxiter=max_iter,
        init=None,
        seed=42
    )
    clust = np.argmax(u, axis=0).astype(float)
    return clust.reshape(shape[0]*shape[1],1).reshape(shape[0],shape[1])

In [ ]:
def MyClust(image=pavia['rgb'], , params={'Algorithm': 'KMeans',
                                         'ImType': 'RGB',
                                         'NumClusts': 3}):
    inp_image = image
    shape = inp_image.shape
    if params['Imtype'].upper() == 'HYPER':
        # TODO Add this to appconfig
        n_components = 30
        pca = PCA(n_components=n_components, svd_solver='auto').fit(hyper)
        inp_image = pca.transform(in_image).reshape(shape[0],shape[1],n_components)
    # TODO Add logic for determining number of clusters 
    if params['Algorithm'].upper() == 'KMEANS':
        out_labels = k_means_clustering(inp=in_image, n_clusters=5)   
    elif params['Algorithm'].upper() == 'FCM':
        #TODO Add the parameter to appconfig
        out_labels = fuzzy_c_means(inp_image=inp_image, n_clusters=2,fuzz_degree=2,error=0.005,max_iter=1000)
    elif params['Algorithm'].upper() == 'SOM':    
        # TODO Add Implementation 
        pass 
    elif params['Algorithm'].upper() == 'SPECTRAL':
        # TODO Add Implementation 
        pass 
    else:
        print('Invalid Algorithm Entered')
        return 
    